In [1]:
import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar

!pip3 install atari-py
!python -m atari_py.import_roms ROMS
!pip install "gym[atari]" "gym[accept-rom-license]" atari_py
!pip install -U "ray[rllib]==1.11"

In [2]:
pip install pygame

In [3]:
pip install torch gym


In [4]:
  # pip install zlib1g-dev cmake
!apt install swig cmake libopenmpi-dev zlib1g-dev
!pip install stable-baselines[mpi]==2.8.0 box2d box2d-kengz

In [5]:
pip install cmake


In [6]:
from torch import nn
import torch
import gym
from collections import deque
import itertools
import numpy as np
import random


In [7]:
import os

import torch.distributed.rpc
from torch import nn
import torch
import gym
from collections import deque
import itertools
import numpy as np
import random

from torch.utils.tensorboard import SummaryWriter

from baselines_wrappers import DummyVecEnv, SubprocVecEnv, Monitor
from pytorch_wrappers import make_atari_deepmind, BatchedPytorchFrameStack, PytorchLazyFrames

import msgpack
from msgpack_numpy import patch as msgpack_numpy_patch
msgpack_numpy_patch()


GAMMA=0.99
BATCH_SIZE=32
BUFFER_SIZE=int(le6) 
MIN_REPLAY_SIZE=1000
EPSILON_START=1.0
EPSILON_END=0.1
EPSILON_DECAY=int(le6)
TARGET_UPDATE_FREQ = 10000
NUM_ENVS = 4
LR = 2.5e-4
SAVE_PATH = './atari_model.pack'
SAVE_INTERVAL = 10000
LOG_DIR ='./logs/atari'
LOG_INTERVAL= 1000

def nature_cnn(observastion_space, depths = (32, 64, 64), final_layer =512):  #from the doc Nature journal
    n_input_channels = observastion_space.shape[0]

    cnn = nn.Sequential(
        nn.Conv2d(n_input_channels, depths[0], kernel_size=8, stride=4),
        nn.ReLU(),
        nn.Conv2d(depths[0], depths[1], kernel_size=4, stride=2),
        nn.ReLU(),
        nn.Conv2d(depths[1], depths[2], kernel_size=3, stride=1),
        nn.ReLU(),
        nn.Flatten())

    with torch.no_grad():
        n_flatten = cnn(torch.as_tensor(observastion_space.sample()[None]).float()).shape[1]  #forward pass through cnn

    out = nn.Sequential(cnn, nn.Linear(n_flatten, final_layer), nn.ReLU())
    return out


class Network(nn.Module):
    def __init__(self, env, device):
        super().__init__()

        self.num_actions = env.action_space.n
        self.device = device
        #in_features = int(np.prod(env.observation_space.shape))  #input is a 3d tensor

        conv_net = nature_cnn(env.observation_space.shape)

        self.net = nn.Sequential(conv_net, Linear(512, self.num_actions))

    def forward(self, x):
        return self.net(x)

    def act(self, obses , epsilon):
        obs_t = torch.as_tensor(obses, dtype=torch.float32, device = self.device)
        q_values = self(obses_t)
        max_q_indices = torch.argmax(q_values, dim=1)[0]
        actions = max_q_indices.detach().tolist()

        for i in range(len(actions)):
            rnd_sample = random.random()
            if rnd_sample <= epsilon:
                actions[i] =random.randint(0, self.num_actions - 1)

        return actions

    def compute_loss(self, transitions, target_net):
        obses = [t[0] for t in transitions]
        actions = np.asarray([t[1] for t in transitions])
        rews = np.asarray([t[2] for t in transitions])
        dones = np.asarray([t[3] for t in transitions])
        new_obses = [t[4] for t in transitions]

        if isinstance(obses[0], PytorchLazyFrames):
            obses = np.stack([o.get_frames() for o in obses])  # converts into numpy array
            new_obses = np.stack([o.get_frames() for o in obses])
        else:
            obses = np.asarray(obses)
            new_obses = np.asarray(new_obses)


        obses_t = torch.as_tensor(obses, dtype=torch.float32, device = self.device)
        actions_t = torch.as_tensor(actions, dtype=torch.int64, device = self.device).unsqueeze(-1)
        rews_t = torch.as_tensor(rews, dtype=torch.float32, device = self.device).unsqueeze(-1)
        dones_t = torch.as_tensor(dones, dtype=torch.float32, device = self.device).unsqueeze(-1)
        new_obses_t = torch.as_tensor(new_obses, dtype=torch.float32, device = self.device)

        # Compute Targets
        # targets = r + gamma * target q vals * (1 - dones)
        target_q_values = target_net(new_obses_t)
        max_target_q_values = target_q_values.max(dim=1, keepdim=True)[0]

        targets = rews_t + GAMMA * (1 - dones_t) * max_target_q_values

        # Compute Loss
        q_values = self(obses_t)
        action_q_values = torch.gather(input=q_values, dim=1, index=actions_t)

        loss = nn.functional.smooth_l1_loss(action_q_values, targets)
        return loss


    def save(self, save_path ):
        params = {k: t.detach().cpu().numpy() for k,t in self.state_dict().items()}    #datas with numpy array rather than tensors
        params_data = msgpack.dumps(params)

        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        with open (save_path, 'wb') as f: #open as binary data
            f.write(params_data)

    def load(self, load_path):
        if not os.path.exists(load_path):
            raise FileNotFoundError(load_path)
        with open(load_path, 'rb') as f:
            params_numpy = msgpack.loads(f.read())

        params = {k: torch.as_tensor(v, device = self.device) for k,v in params_numpy.items()} #transfor to tensor
        self.load_state_dict(params)#loading


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#env = gym.make('CartPole-v0')
make_env =lambda: Monitor(make_atari_deepmind('Breakout-v0'), allow_early_resets = True)

vec_env = DummyVecEnv([make_env for _ in range(NUM_ENVS)]) #runs env in sequence
#env = SubprocVecEnv([make_env for _ in NUM_ENVS]) #each of the env in parallel

env = BatchedPytorchFrameStack(vec_env, k=4)

replay_buffer = deque(maxlen=BUFFER_SIZE)
epinfos_buffer = deque([], maxlen=100)

episode_count = 0

summary_writer = SummaryWriter(LOG_DIR)

online_net = Network(env, device= device)
target_net = Network(env, device = device)

online_net = online_net.to(device)
online_net = target_net.to(device)

target_net.load_state_dict(online_net.state_dict())

optimizer = torch.optim.Adam(online_net.parameters(), lr=5e-4)

# Initialize replay buffer
obs = env.reset()
for _ in range(MIN_REPLAY_SIZE):
    actions = [env.action_space.sample() for _ in range(NUM_ENVS)] #one  for each env, multiple actions sampling

    new_obses, rews, dones, _ = env.step(actions)
    for obs, action, rew, done, new_pbs in zip(obses, actions, rews, dones, new_obses): #zipping components of our transitions with batch dim
        transition = (obs, action, rew, done, new_obs) #pulling each ele to construct the single transition separately
        replay_buffer.append(transition) #append to replay buffer
        obses = new_obses
        #after if its done, DummyVecEnv/SubprocVecEnv resets

    obses = new_obses
# Main Training Loop
obses = env.reset()
for step in itertools.count():
    epsilon = np.interp(step*NUM_ENVS, [0, EPSILON_DECAY], [EPSILON_START, EPSILON_END])

    rnd_sample = random.random()

    if isinstance(obses[0], PytorchLazyFrames):
        act_obses = np.stack([o.get_frames() for o in obses]) #converts into numpy array
        actions = online_net.act(act_obses, epsilon)
    else:
        actions = online_net.act(obses, epsilon)

    new_obses, rews, dones, infos = env.step(actions)
    for obs, action, rew, done, new_obs, info in zip(obses, actions, rews, dones,new_obses, infos):  # zipping components of our transitions with batch dim
        transition = (obs, action, rew, done, new_obs)  # pulling each ele to construct the single transition separately
        replay_buffer.append(transition)  # append to replay buffer
        obses = new_obses
        # after if its done, DummyVecEnv/SubprocVecEnv resets

        if done:
            epinfos_buffer.append(info['episode'])
            episode_count += 1

    obses = new_obses

    #watch agent play


    #gradient desc.
    transitions = random.sample(replay_buffer, BATCH_SIZE)
    loss = online_net.compute_loss(transitions, target_net)

    # Gradient Descent
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Update Target Net
    if step % TARGET_UPDATE_FREQ == 0:
        target_net.load_state_dict(online_net.state_dict())

    # Logging
    if step % LOG_INTERVAL == 0:
        rew_mean = np.mean([e['r'] for e in epinfos_buffer]) or 0 #mean of all rewards iunside the episode buffer
        len_mean = np.mean([e['l '] for e in epinfos_buffer]) or 0 #length of episode
        print()
        print('Step:', step)
        print('Avg rew:', rew_mean)
        print('Av Ep Len:', len_)
        print('Episodes:', episode_count)

        summary_writer.add_scalar('Av reward', rew_mean, global_step=step)
        summary_writer.add_scalar('AvEpLen', len_mean, global_step=step)
        summary_writer.add_scalar('Episodes', episode_count, global_step=step)
    #SAVING
    if step % SAVE_INTERVAL == 0 and step !=0:
        print('Saving..')
        online_net.save(SAVE_PATH)

